# Friends Eng

라이브러리 import

In [1]:
import pandas as pd
#import torch
#from torch.nn import functional as F
#from torch.utils.data import DataLoader, Dataset
!pip install transformers
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
#from tqdm.notebook import tqdm
#!pip install -U pandas-profiling
#import pandas_profiling
import json

     |████████████████████████████████| 1.5MB 5.8MB/s 
     |████████████████████████████████| 890kB 36.1MB/s 
     |████████████████████████████████| 2.9MB 42.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=e871e1dbf78f82340d77a3be07f4e6c834e7eb03e0463d506a54f5d527d3694f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import numpy as np
from nltk.tokenize import sent_tokenize
#from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords  
nltk.download('stopwords')
from xgboost import XGBClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


데이터 읽기 : 코드 참고 https://blog.naver.com/huje89/222138858861

In [5]:
with open('friends_train.json') as json_file : 
  json_train = json.load(json_file)
  train_data = []
for rows in json_train : 
    for row in rows : 
      train_data.append([row['annotation'], row['speaker'], row['utterance'], row['emotion']])
train_f = pd.DataFrame(train_data, columns = ['annotation', 'speaker', 'utterance', 'emotion']) 

with open('friends_test.json') as json_file : 
  json_train = json.load(json_file)
  test_data = []
for rows in json_train : 
    for row in rows : 
      test_data.append([row['annotation'], row['speaker'], row['utterance'], row['emotion']])
test_f = pd.DataFrame(test_data, columns = ['annotation', 'speaker', 'utterance', 'emotion']) 

텍스트 분석 : 코드 참고 https://colab.research.google.com/drive/18nAZ_uqxP5pb8hKBHUc-UuTapAzdcqgE#scrollTo=f8hRnRUG_ss8

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train_f['utterance'])

In [8]:
print(X_train_counts.shape)
print(X_train_counts[0])
print(count_vect.vocabulary_)

(10561, 5444)
  (0, 182)	1
  (0, 5203)	1
  (0, 4806)	2
  (0, 3651)	1
  (0, 3539)	1
  (0, 3353)	1
  (0, 3163)	1
  (0, 1027)	1
  (0, 4952)	1
  (0, 1917)	1
  (0, 2605)	1
  (0, 4886)	1
  (0, 2054)	1
  (0, 4714)	1
{'also': 182, 'was': 5203, 'the': 4806, 'point': 3651, 'person': 3539, 'on': 3353, 'my': 3163, 'company': 1027, 'transition': 4952, 'from': 1917, 'kl': 2605, 'to': 4886, 'gr': 2054, 'system': 4714, 'you': 5418, 'must': 3158, 've': 5121, 'had': 2121, 'your': 5422, 'hands': 2147, 'full': 1927, 'that': 4804, 'did': 1330, 'so': 4382, 'let': 2725, 'talk': 4728, 'little': 2786, 'bit': 504, 'about': 80, 'duties': 1495, 'all': 168, 'right': 3986, 'now': 3285, 'll': 2791, 'be': 420, 'heading': 2193, 'whole': 5285, 'division': 1380, 'have': 2184, 'lot': 2830, 'of': 3323, 'see': 4177, 'but': 703, 'there': 4816, 'perhaps': 3532, '30': 34, 'people': 3523, 'under': 5055, 'can': 744, 'dump': 1489, 'certain': 828, 'amount': 197, 'them': 4809, 'good': 2030, 'know': 2622, 'we': 5223, 'go': 2015, 'i

In [9]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=True).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)

In [10]:
print(X_train_tf.shape)
print(X_train_tf[0])

(10561, 5444)
  (0, 5203)	0.15777706576202713
  (0, 4952)	0.36105528210912236
  (0, 4886)	0.11223370960779906
  (0, 4806)	0.2190054956559119
  (0, 4714)	0.3138004978885935
  (0, 3651)	0.3003465581277965
  (0, 3539)	0.2706055908673653
  (0, 3353)	0.1530862630151083
  (0, 3163)	0.14300302933454048
  (0, 2605)	0.36105528210912236
  (0, 2054)	0.36105528210912236
  (0, 1917)	0.20989686688603945
  (0, 1027)	0.31961513106012945
  (0, 182)	0.26732348248151927


In [11]:
#Baseline Model 
from sklearn.pipeline import Pipeline
pipe_clf = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2), tokenizer = nltk.word_tokenize)),
    #('tfidf', TfidfTransformer()),
    ('clf', XGBClassifier(colsample_bylevel=0.8, olsample_bytree=0.8,max_depth=8,learning_rate=0.2)),
])
pipe_clf.fit(train_f['utterance'], train_f['emotion'])

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function word_token...
                               colsample_bylevel=0.8, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.2,
                               max_delta_step=0, max_depth=8,
                               min_child_weight=1, missing=None,
                               n_estimators=100, n_jobs=

In [12]:
predicted = pipe_clf.predict(test_f['utterance'])
result = np.mean(predicted == test_f['emotion'])
print(result)

0.5875542691751086


In [13]:
print(predicted)

['neutral' 'non-neutral' 'neutral' ... 'joy' 'neutral' 'neutral']


In [14]:
from collections import Counter
print(Counter(train_f['emotion']))
print(Counter(test_f['emotion']))

Counter({'neutral': 4752, 'non-neutral': 2017, 'joy': 1283, 'surprise': 1220, 'anger': 513, 'sadness': 351, 'disgust': 240, 'fear': 185})
Counter({'neutral': 1287, 'non-neutral': 541, 'joy': 304, 'surprise': 286, 'anger': 161, 'sadness': 85, 'disgust': 68, 'fear': 32})


In [15]:
from sklearn import metrics

result = metrics.classification_report(
    y_true = test_f['emotion'],
    y_pred = predicted)#,
    #target_names = test_f['emotion'])
print(result)

              precision    recall  f1-score   support

       anger       0.49      0.14      0.22       161
     disgust       0.50      0.03      0.06        68
        fear       0.00      0.00      0.00        32
         joy       0.45      0.57      0.50       304
     neutral       0.67      0.90      0.77      1287
 non-neutral       0.35      0.23      0.28       541
     sadness       0.44      0.16      0.24        85
    surprise       0.57      0.44      0.50       286

    accuracy                           0.59      2764
   macro avg       0.44      0.31      0.32      2764
weighted avg       0.55      0.59      0.54      2764



# 리더보드 테스트

In [ ]:
k_test = pd.read_csv('en_data.csv')
k_test.head()

In [ ]:
predicted = pipe_clf.predict(k_test['utterance'])
print(predicted)

In [ ]:
pd.DataFrame(predicted).to_csv("friends_2020511028.csv")